In [1]:
# 나라별 유명 검색 사이트에서 선택한 나라들의 탑 순위 및 원하는 검색들의 결과 보여주는. 

# naver.com네이버 - 한국 
# https://tieba.baidu.com/index.html - 중국
# https://www.yahoo.com/ - 미국
# https://www.yahoo.co.jp/ - 일본

# 위의 각 검색사이트의 현재 검색순위1~10등, 그리고 그것을 구글에서 검색한 이미지를 같이 보여준다.
# cvc파일로 저장한다.
# 중국 미국 일본에서 검색한 것은 번역기로 번역한 것도 같이 보여준다.
# 원문을 컴퓨터가 읽어주기.


In [4]:
pip install gTTS

  Stored in directory: C:\Users\student\AppData\Local\pip\Cache\wheels\ac\d3\52\db6c154b20dfaab7e0b514eb5eef92cecd057e40e16fdda58b
  Stored in directory: C:\Users\student\AppData\Local\pip\Cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
  Stored in directory: C:\Users\student\AppData\Local\pip\Cache\wheels\dd\11\61\33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
Successfully built gTTS bs4 gtts-token
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from gtts import gTTS
from selenium import webdriver
new_ul_naver = []
new_ul_yahoo = []
naver_list = []
baidu_list = []
yahoo_list = []
new_ul_yahoo = []
yahoo_jp_list = []
rank = []

In [11]:
#Naver searching
NAVER_URL = "https://www.naver.com"
resp_naver = requests.get(NAVER_URL)
soup_naver = BeautifulSoup(resp_naver.content, 'html.parser')
ul_contents_naver = soup_naver.find('div', class_='ah_roll_area PM_CL_realtimeKeyword_rolling').find('ul')
for content in ul_contents_naver.contents:
    if not str(content).strip():
        continue
    new_ul_naver.append(content)
for li in range(0,10):
    a_tag = new_ul_naver[li].find('a').find('span')
    naver_list.append(a_tag.find_next().text)

In [12]:
#Baidu searching
BAIDU_URL = "https://tieba.baidu.com/hottopic/browse/topicList"
resp = requests.get(BAIDU_URL)
temp1 = resp.json()
temp2 = temp1.get("data")
temp3 = temp2.get("bang_topic")
temp4 = temp3.get("topic_list")
for counting in range(0,10):
    temp5 = temp4[counting]
    baidu_list.append(temp5.get("topic_name"))

In [13]:
#Yahoo searching
YAHOO_URL = "https://www.yahoo.com/"
resp = requests.get(YAHOO_URL)
soup = BeautifulSoup(resp.content, 'html.parser')
ul_contents = soup.find('ul', class_='Pos(r) Mt(10px)').find('li').find('ul')
for content in ul_contents.contents:
    if not str(content).strip():
        continue
    new_ul_yahoo.append(content)
for li in new_ul_yahoo:
    a_tag = li.find('a').find('span')
    yahoo_list.append(a_tag.find_next().text)

In [14]:
#yahoo_jp searching
driver = webdriver.Chrome("C:\\Users\\student\\Downloads\\chromedriver_win32 (2)\\chromedriver.exe")
driver.get("https://search.yahoo.co.jp/realtime")

for x in range(1,3):
    for y in range(1,6):
        daily1 = 'daily_raking_' + str(5*(x-1) + y)
        rank.append(daily1)
        rank[(5*(x-1) + y - 1)] = driver.find_element_by_xpath('//*[@id="Te"]/div[2]/div[' + str(x) + ']/div[' + str(y) + ']/p/a')
        yahoo_jp_list.append(rank[(5*(x-1) + y - 1)].text)

In [15]:
from time import sleep
def searching_images_and_save(address, name_list, image_ad, start_from, image_ad2, saving_name):
    for listing in range(0,10):
        driver.get(address+ name_list[listing])
        driver.implicitly_wait(10)
        for counting in range(0,4):
            elem = driver.find_element_by_xpath(image_ad+ str(counting+start_from) + image_ad2)
            elem.screenshot_as_png
            filename = saving_name+ str(listing)+ str(counting) +'.png'
            with open(filename, 'wb') as f:
                f.write(elem.screenshot_as_png)
                sleep(1)
                
                

# searching_images_and_save("https://search.yahoo.co.jp/image/search?p=", yahoo_jp_list, '//*[@id="gridlist"]/div[', 1, ']','yahoo_jp_img')
searching_images_and_save("https://images.search.yahoo.com/search/images?p=", yahoo_list, '//*[@id="resitem-', 0, '"]', 'yahoo_img')
searching_images_and_save("http://image.baidu.com/search/index?tn=baiduimage&word=", baidu_list, '//*[@id="imgid"]/div/ul/li[', 1, ']', 'baidu_img')
searching_images_and_save("https://search.naver.com/search.naver?where=image&query=", naver_list, '//*[@id="_sau_imageTab"]/div[2]/div[2]/div[', 1, ']', 'naver_img')

In [16]:
def reading_list(name_list, lg, name):
    for ct in range(0,10):
        text = name_list[ct]
        tts = gTTS(text=text, lang= "{}".format(lg))
        tts.save(str(name) + str(ct) +".mp3")
        
reading_list(yahoo_list, 'en', 'yahoo')
reading_list(naver_list, 'ko', 'naver')
reading_list(baidu_list, 'zh-cn', 'baidu')
reading_list(yahoo_jp_list, 'ja', 'yahoo_jr')


In [17]:
tr_jp = []
tr_y = []
tr_b = []

In [18]:
for ct in range(0,10):
    driver.get("https://translate.google.co.kr/?hl=ko")
    driver.implicitly_wait(10)
    search_elem = driver.find_element_by_xpath('//*[@id="source"]')
    search_elem.send_keys(yahoo_jp_list[ct])
    elem = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/div[3]/div[1]/div[2]/div/span[1]/span[1]')
    tr_jp.append(elem.text)
for ct in range(0,10):
    driver.get("https://translate.google.co.kr/?hl=ko")
    driver.implicitly_wait(10)
    search_elem = driver.find_element_by_xpath('//*[@id="source"]')
    search_elem.send_keys(yahoo_list[ct])
    elem = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/div[3]/div[1]/div[2]/div/span[1]/span[1]')
    tr_y.append(elem.text)
for ct in range(0,10):
    driver.get("https://translate.google.co.kr/?hl=ko")
    driver.implicitly_wait(10)
    search_elem = driver.find_element_by_xpath('//*[@id="source"]')
    search_elem.send_keys(baidu_list[ct])
    elem = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/div[3]/div[1]/div[2]/div/span[1]/span[1]')
    tr_b.append(elem.text)

In [20]:
df = pd.DataFrame({'Naver': naver_list, 'Baidu' : baidu_list, 'Yahoo': yahoo_list, 'Yahoo_jp': yahoo_jp_list, 'tr_jp': tr_jp, 'tr_y': tr_y, 'tr_b': tr_b},index = range(1,11))

In [21]:
df

,Naver,Baidu,Yahoo,Yahoo_jp,tr_jp,tr_y,tr_b
1,비아이,柳岩摔倒,Laura Dern,PCエンジン,PC 엔진,로라 데른,리우 얀이 쓰러졌다.
2,전선혜,彩礼要一万八,Khatia Buniatishvili,核戦争防止国際医師会議,핵전쟁 방지 국제 의사 회,Khatia Buniyatishvili,신부 가격은 1 만 8 천
3,u20 결승,中国女排横扫波兰女排,Marianne Williamson,全裸監督,완전한 알몸 감독,마리안 윌리엄슨,"폴란드 여자 대표팀, 중국 여자 배구"
4,축구 결승,男子用牛油果抢银行,Marisa Tomei,たぬき開発,너구리,마리사,아보카도로 도둑질하는 남자
5,다니엘 튜더,快递员遭恶意投诉下跪,Kristin Cavallari,加藤シゲアキ,카토 시게아키,크리스틴 카발라리,택배가 악의적으로 불평을 받았다.
6,유권,男婴抚养27年后续,Termite Control,ペイルライダー,페일,흰개미 제어,27 년간 아기 양육
7,우크라이나,动物管理局带火朱雀吧,Anna Faris,幹子,幹子,나는 기사 야.,화재 Suzaku와 동물 행정
8,한국 우크라이나,寒门还能出贵子吗,Online MBA Degree,任天堂,닌텐도,온라인 MBA 학위,차가운 문에는 여전히 아이가있을 수 있습니까?
9,유권 전선혜,找不到工作告母校,Senior Dating Sites,斉藤朱夏,사이토,수석 데이트 사이트,어머니의 학교에서 일자리를 찾을 수 없습니다.
10,2019 u20 월드컵 결승,秦明生死语者,Carli Lloyd,ルリナ,루리나,칼리 로이드,진 Mingsheng의 죽음


In [22]:
df.to_csv("temp.csv", encoding="utf8")
df.to_excel("temp1.xlsx")

### 번외. 셀레니움 감추기

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome("C:\\Users\\dty\\Downloads\\chromedriver_win32\\chromedriver", options=options)
driver.get('https://www.google.com/doodles')

print('Title: "{}"'.format(driver.title))
driver.quit()